In [1]:
!pip install surprise

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [3]:
import os
print(os.getcwd())

C:\Users\w1305


In [4]:
os.chdir(r"C:\Users\w1305\Downloads")
print(os.getcwd())

C:\Users\w1305\Downloads


In [5]:
import json
import pandas as pd

# 读取 JSON Lines 格式的文件
json_file_path = "games_metadata.json"
metadata_list = []

with open(json_file_path, "r", encoding="utf-8") as file:
    for line in file:
        try:
            metadata_list.append(json.loads(line.strip()))  # 解析 JSON
        except json.JSONDecodeError:
            continue  # 跳过解析失败的行

# 转换为 Pandas DataFrame
metadata_df = pd.DataFrame(metadata_list)

# 只保留需要的字段
metadata_df = metadata_df[["app_id", "tags"]]

# 将 tags 转换为字符串格式（避免 NaN）
metadata_df["tags"] = metadata_df["tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")

In [6]:
steam_data = pd.read_csv("steamgame.csv")
steam_data["app_id"] = steam_data["app_id"].astype(int)
metadata_df["app_id"] = metadata_df["app_id"].astype(int)

# 合并游戏数据和标签数据（基于 app_id）
steam_data = steam_data.merge(metadata_df, on="app_id", how="left")

# 处理 tags 列，填充 NaN 并转换为字符串
steam_data["tags"] = steam_data["tags"].fillna("").astype(str)

In [7]:
# Load datasets
users_df = pd.read_csv("users.csv")
recommendations_df = pd.read_csv("recommendations.csv")

In [8]:
steam_data.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,tags
0,13500,Prince of Persia: Warrior Within™,11/21/2008,True,False,False,Very Positive,84,2199,9.99,9.99,0,True,Action Adventure Parkour Third Person Great So...
1,22364,BRINK: Agents of Change,8/3/2011,True,False,False,Positive,85,21,2.99,2.99,0,True,Action
2,113020,Monaco: What's Yours Is Mine,4/24/2013,True,True,True,Very Positive,92,3722,14.99,14.99,0,True,Co-op Stealth Indie Heist Local Co-Op Strategy...
3,226560,Escape Dead Island,11/18/2014,True,False,False,Mixed,61,873,14.99,14.99,0,True,Zombies Adventure Survival Action Third Person...
4,249050,Dungeon of the ENDLESS™,10/27/2014,True,True,False,Very Positive,88,8784,11.99,11.99,0,True,Roguelike Strategy Tower Defense Pixel Graphic...


In [9]:
recommendations_df.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4


In [10]:
print(recommendations_df["hours"].describe())

count    4.115479e+07
mean     1.006022e+02
std      1.761675e+02
min      0.000000e+00
25%      7.800000e+00
50%      2.730000e+01
75%      9.920000e+01
max      1.000000e+03
Name: hours, dtype: float64


In [11]:
users_df.head()

,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2


In [12]:
# Merge recommendations with game details
recommendations_games = recommendations_df.merge(steam_data, on="app_id", how="left")

# Merge recommendations with user details
recommendations_full = recommendations_games.merge(users_df, on="user_id", how="left")
print(recommendations_full)

           app_id  helpful  funny        date  is_recommended  hours  \
0          975370        0      0  2022-12-12            True   36.3   
1          304390        4      0  2017-02-17           False   11.5   
2         1085660        2      0  2019-11-17            True  336.5   
3          703080        0      0  2022-09-23            True   27.4   
4          526870        0      0  2021-01-10            True    7.9   
...           ...      ...    ...         ...             ...    ...   
41154789   633230        0      0  2021-02-15            True   41.0   
41154790   758870        8      0  2019-07-18           False    8.0   
41154791   696170        3     10  2018-03-26           False    2.0   
41154792   696170        0      0  2018-06-11            True    4.0   
41154793  1089980        2      0  2020-09-16            True   14.0   

           user_id  review_id                              title date_release  \
0            51580          0                     Dwar

In [13]:
print(recommendations_full.shape)
print(recommendations_full[['user_id', 'app_id']].isnull().sum())
print(recommendations_full.dtypes)

(41154794, 23)
user_id    0
app_id     0
dtype: int64
app_id              int64
helpful             int64
funny               int64
date               object
is_recommended       bool
hours             float64
user_id             int64
review_id           int64
title              object
date_release       object
win                  bool
mac                  bool
linux                bool
rating             object
positive_ratio      int64
user_reviews        int64
price_final       float64
price_original    float64
discount            int64
steam_deck           bool
tags               object
products            int64
reviews             int64
dtype: object


In [14]:
unique_ratings = recommendations_full["rating"].unique()
print(unique_ratings)

['Overwhelmingly Positive' 'Mixed' 'Very Positive' 'Mostly Positive'
 'Positive' 'Mostly Negative' 'Overwhelmingly Negative' 'Negative'
 'Very Negative']


In [15]:
# 选取最活跃的 10,000 名用户
top_users = recommendations_full["user_id"].value_counts().head(5000).index

# 选取最常被游玩的 10,000 款游戏
top_games = recommendations_full["app_id"].value_counts().head(5000).index

# 过滤数据
filtered_df = recommendations_full[
    (recommendations_full["user_id"].isin(top_users)) & 
    (recommendations_full["app_id"].isin(top_games)) &
    (recommendations_full["hours"] > 1)  # 去掉游玩时间小于 1 小时的
]

# 显示缩小后的数据规模
print(filtered_df)

           app_id  helpful  funny        date  is_recommended  hours  \
422        435150       83      0  2017-12-15            True  199.9   
425        275850        0      0  2020-10-10            True   13.6   
691        105600        5      3  2014-09-28            True  911.8   
712       1794680        0      0  2022-02-19            True   77.3   
1180       686810        6      0  2020-07-20            True   67.4   
...           ...      ...    ...         ...             ...    ...   
41154313   203160        2      0  2014-01-29            True   10.0   
41154516   979640        3      0  2021-08-16           False   10.0   
41154723   897450        0      0  2020-11-12            True    6.0   
41154771   362960        0      0  2019-07-06            True   35.0   
41154784   633230        0      0  2021-06-21           False   20.0   

           user_id  review_id                                          title  \
422        6334645        422  Divinity: Original Sin 2

In [16]:
rating_map = {
    "Overwhelmingly Positive": 4,
    "Very Positive": 3,
    "Positive": 2,
    "Mostly Positive": 1,
    "Mixed": 0,  # 中立归为 0
    "Negative": -1,
    "Mostly Negative": -2,
    "Very Negative": -3,
    "Overwhelmingly Negative": -4
}

filtered_df.loc[:, "rating_score"] = filtered_df["rating"].map(rating_map)

C:\Users\w1305\AppData\Local\Temp\ipykernel_8076\3069529864.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, "rating_score"] = filtered_df["rating"].map(rating_map)


In [17]:
from surprise import Dataset, Reader
filtered_df.loc[:, "weighted_score"] = 0.3 * filtered_df["hours"] + 0.7 * filtered_df["rating_score"]


filtered_df.loc[:, "weighted_score"] = filtered_df["weighted_score"].fillna(filtered_df["hours"] * 0.7)

# 定义 Surprise 的评分范围
reader = Reader(rating_scale=(0, filtered_df["weighted_score"].max()))
data = Dataset.load_from_df(filtered_df[["user_id", "app_id", "weighted_score"]], reader)

C:\Users\w1305\AppData\Local\Temp\ipykernel_8076\154572362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, "weighted_score"] = 0.3 * filtered_df["hours"] + 0.7 * filtered_df["rating_score"]


In [18]:
from surprise import SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy

# 拆分训练集 & 测试集
trainset, testset = train_test_split(data, test_size=0.2)

# 训练 SVD 模型
svd_model = SVD()
svd_model.fit(trainset)

# 评估 SVD
predictions = svd_model.test(testset)
accuracy.rmse(predictions)

RMSE: 291.2294


291.22939610672506

In [19]:
from surprise import KNNBasic

# 选择基于物品的协同过滤
sim_options = {
    "name": "cosine",  # 余弦相似度
    "user_based": False  # 物品（游戏）推荐
}

knn_model = KNNBasic(sim_options=sim_options)
knn_model.fit(trainset)

# 评估 KNNBasic
knn_predictions = knn_model.test(testset)
accuracy.rmse(knn_predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 26.0750


26.07503153021298

In [20]:
def recommend_games_svd(user_id, top_n=10):
    # 找到用户未玩过的游戏
    played_games = filtered_df[filtered_df["user_id"] == user_id]["app_id"].unique()
    all_games = filtered_df["app_id"].unique()
    unplayed_games = [game for game in all_games if game not in played_games]

    # 预测评分
    predictions = [svd_model.predict(user_id, game) for game in unplayed_games]

    # 排序并选择 top N
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_games = [pred.iid for pred in predictions[:top_n]]

    # 获取游戏信息
    recommended_df = filtered_df[filtered_df["app_id"].isin(top_games)][["app_id", "title","tags"]]

    recommended_df = recommended_df.drop_duplicates(subset=["app_id"])
    
    return recommended_df

# 示例：给 user_id = 456 推荐游戏
svd_recommendations = recommend_games_svd(8107668)

display(svd_recommendations)

,app_id,title,tags
422,435150,Divinity: Original Sin 2 - Definitive Edition,
425,275850,No Man's Sky,
691,105600,Terraria,
1180,686810,Hell Let Loose,
1228,1222670,The Sims™ 4,
1552,1092790,Inscryption,
2105,1259420,Days Gone,
2165,1325200,Nioh 2 – The Complete Edition,RPG Action Souls-like Character Customization ...
2491,304390,FOR HONOR™,
3167,397540,Borderlands 3,


In [21]:
def recommend_similar_games_knn(game_id, top_n=10):
    inner_id = knn_model.trainset.to_inner_iid(game_id)  # 转换为 Surprise 内部 ID
    neighbors = knn_model.get_neighbors(inner_id, k=top_n)  # 获取最近邻游戏

    # 转换回原始 ID
    similar_games = [knn_model.trainset.to_raw_iid(n) for n in neighbors]

    # 获取游戏名称
    recommended_dfknn = filtered_df[filtered_df["app_id"].isin(similar_games)][["app_id", "title", "tags"]]
    recommended_dfknn = recommended_dfknn.drop_duplicates(subset=["app_id"])
    return recommended_dfknn

# 示例：获取与某个游戏（app_id = 123）最相似的游戏
knn_recommendations = recommend_similar_games_knn(633230)
display(knn_recommendations)

,app_id,title,tags
11174261,880950,Maitetsu:Pure Station,Sexual Content Anime Visual Novel Mature Nudit...
11298442,916730,Gato Roboto,Indie Metroidvania Action Adventure Pixel Grap...
15262118,331670,The Jackbox Party Pack,Casual Trivia Party Local Multiplayer Funny In...
17140811,243970,Invisible Inc.,Stealth Turn-Based Strategy Cyberpunk Turn-Bas...
18820124,13570,Tom Clancy's Splinter Cell Chaos Theory®,Stealth Action Third Person Classic Co-op Grea...
27803406,717640,Reigns: Her Majesty,Indie Life Sim Female Protagonist Choices Matt...
29319606,115110,Stacking,Adventure Puzzle Indie Comedy Singleplayer Thi...
29446977,932850,Simmiland,Simulation Indie God Game Pixel Graphics Card ...
34760852,994280,古剑奇谭三(Gujian3),RPG Story Rich Martial Arts Action RPG Singlep...
35525467,716500,Eliza,Female Protagonist Visual Novel Emotional Psyc...


In [22]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 过滤数据，确保没有 NaN 值
filtered_df = filtered_df.dropna(subset=["user_id", "app_id", "weighted_score"])

# 初始化编码器
user_encoder = LabelEncoder()
game_encoder = LabelEncoder()

# 先对完整数据集 `fit()`
filtered_df["user_encoded"] = user_encoder.fit_transform(filtered_df["user_id"])
filtered_df["game_encoded"] = game_encoder.fit_transform(filtered_df["app_id"])

# 划分训练 & 测试集
train, test = train_test_split(filtered_df, test_size=0.2, random_state=42, stratify=filtered_df["user_id"])

# 在 `train` 和 `test` 上 `transform()`，保持一致性
train["user_encoded"] = user_encoder.transform(train["user_id"])
test["user_encoded"] = user_encoder.transform(test["user_id"])

train["game_encoded"] = game_encoder.transform(train["app_id"])
test["game_encoded"] = game_encoder.transform(test["app_id"])

# ✅ **提取用户、游戏和评分数据**
train_users = train["user_encoded"].values
train_games = train["game_encoded"].values
train_scores = train["weighted_score"].values  # 🎯 评分数据

test_users = test["user_encoded"].values
test_games = test["game_encoded"].values
test_scores = test["weighted_score"].values  # 🎯 评分数据

In [30]:
train_scores[train_scores < 0] = 0
test_scores[test_scores < 0] = 0

In [32]:
train_scores_log = np.log1p(train_scores)
test_scores_log = np.log1p(test_scores)

In [34]:
train_scores_log[np.isinf(train_scores_log)] = np.nan
test_scores_log[np.isinf(test_scores_log)] = np.nan

In [36]:
train_scores_log = np.nan_to_num(train_scores_log)
test_scores_log = np.nan_to_num(test_scores_log)

In [38]:
print(np.isnan(train_scores_log).sum(), np.isnan(test_scores_log).sum())  
print(np.isinf(train_scores_log).sum(), np.isinf(test_scores_log).sum())  

0 0
0 0


In [62]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 定义嵌入层的维度
embedding_dim = 64  # 维度可以调节，通常 32~128

# 用户输入层
user_input = keras.layers.Input(shape=(1,), name="user_input")
user_embedding = keras.layers.Embedding(
    input_dim=len(user_encoder.classes_), output_dim=embedding_dim, name="user_embedding"
)(user_input)
user_vec = keras.layers.Flatten()(user_embedding)

# 游戏输入层
game_input = keras.layers.Input(shape=(1,), name="game_input")
game_embedding = keras.layers.Embedding(
    input_dim=len(game_encoder.classes_), output_dim=embedding_dim, name="game_embedding"
)(game_input)
game_vec = keras.layers.Flatten()(game_embedding)

# 合并用户 & 游戏特征
merged = keras.layers.Concatenate()([user_vec, game_vec])
hidden = keras.layers.Dense(128, activation="relu")(merged)
hidden = Dropout(0.2)(hidden)
hidden = keras.layers.Dense(64, activation="relu")(hidden)
output = keras.layers.Dense(1, activation="linear")(hidden)  # 预测评分

# 构建模型
model = keras.models.Model([user_input, game_input], output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# 打印模型结构
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ game_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_embedding (Embedding)    │ (None, 1, 64)             │         320,000 │ user_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ game_embedding (Embedding)    │ (None, 1, 64)             │         319,424 │ game_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_6 (Flatten)           │ (None, 64)                │               0 │ user_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_7 (Flatten)           │ (None, 64)                │               0 │ game_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_3 (Concatenate)   │ (None, 128)               │               0 │ flatten_6[0][0],           │
│                               │                           │                 │ flatten_7[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 128)               │          16,512 │ concatenate_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 64)                │           8,256 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 1)                 │              65 │ dense_8[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 664,257 (2.53 MB)

 Trainable params: 664,257 (2.53 MB)

 Non-trainable params: 0 (0.00 B)

In [64]:
# 训练数据
train_users = train["user_encoded"].values
train_games = train["game_encoded"].values
train_scores = train["weighted_score"].values

# 测试数据
test_users = test["user_encoded"].values
test_games = test["game_encoded"].values
test_scores = test["weighted_score"].values

# 训练模型
history = model.fit(
    [train_users, train_games], train_scores,
    validation_data=([test_users, test_games], test_scores),
    epochs=10, batch_size=512, verbose=1
)

Epoch 1/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 632.3781 - mae: 10.1681 - val_loss: 516.9658 - val_mae: 8.9383
Epoch 2/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 498.0192 - mae: 8.8951 - val_loss: 515.5370 - val_mae: 8.8280
Epoch 3/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 478.2205 - mae: 8.6902 - val_loss: 518.7718 - val_mae: 8.9157
Epoch 4/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 475.2315 - mae: 8.6532 - val_loss: 521.7332 - val_mae: 8.9660
Epoch 5/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 459.4306 - mae: 8.4941 - val_loss: 523.3442 - val_mae: 8.9166
Epoch 6/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 450.4503 - mae: 8.4364 - val_loss: 527.1870 - val_mae: 8.9978
Epoch 7/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 441.2383 - mae: 8.3251 - val_loss: 536.9070 - val_mae: 9.0603
Epoch 8/10
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 438.2504 - mae: 8.2544 - val_loss: 531.4382 - val_mae: 8.9965
Epoch 9

In [80]:
def recommend_games_nn(user_id, top_n=10):
    user_id = int(user_id)

    user_idx = user_encoder.transform([user_id])[0]

    all_games = np.array(range(len(game_encoder.classes_)))
    predicted_scores = model.predict([np.array([user_idx] * len(all_games)), all_games])

    top_games_idx = np.argsort(predicted_scores.flatten())[::-1][:top_n]
    top_games_ids = game_encoder.inverse_transform(top_games_idx)

    recommended_df = filtered_df[filtered_df["app_id"].isin(top_games_ids)][["app_id", "title", "tags"]]
    recommended_df = recommended_df.drop_duplicates(subset=["app_id"]).head(top_n)

    return recommended_df

# 示例：给 user_id =  推荐游戏
nn_recommendations = recommend_games_nn(8107668)

display(nn_recommendations)

156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,app_id,title,tags
7771,271590,Grand Theft Auto V,
10510,4000,Garry's Mod,
14371,730,Counter-Strike: Global Offensive,
85447,39210,FINAL FANTASY XIV Online,
98484,359550,Tom Clancy's Rainbow Six® Siege,
152202,570,Dota 2,
6201900,1454400,Cookie Clicker,Clicker Idler Casual Indie Pixel Graphics Mana...
30079508,665300,Stream Avatars,Video Production Animation & Modeling Utilitie...
31690699,1094780,Fisher Online,Simulation Massively Multiplayer Fishing Early...
33257645,1009850,OVR Advanced Settings,VR Utilities Free to Play Software Indie


In [78]:
def recommend_similar_games_nn(app_id, top_n=10):
    app_id = int(app_id)
    
    # **获取该游戏的索引**
    game_idx = game_encoder.transform([app_id])[0]
    
    # **提取神经网络中的 `game_embedding` 层**
    game_embedding_layer = model.get_layer("game_embedding")
    game_embedding_weights = game_embedding_layer.get_weights()[0]  # (num_games, embedding_dim)

    # **获取目标游戏的 embedding 向量**
    target_game_vector = game_embedding_weights[game_idx].reshape(1, -1)  # (1, embedding_dim)

    # **计算所有游戏的 embedding**
    all_game_vectors = game_embedding_weights  # (num_games, embedding_dim)

    # **计算与其他游戏的 L2 距离**
    distances = np.linalg.norm(all_game_vectors - target_game_vector, axis=1)

    # **获取距离最近的 Top-N 游戏（排除自身）**
    top_games_idx = np.argsort(distances)[1:top_n+1]  # 排除自身（索引 0）

    # **转换回游戏 ID**
    top_games_ids = game_encoder.inverse_transform(top_games_idx)

    # **获取推荐游戏信息**
    recommended_df = filtered_df[filtered_df["app_id"].isin(top_games_ids)][["app_id", "title", "tags"]]
    recommended_df = recommended_df.drop_duplicates(subset=["app_id"]).head(top_n)

    return recommended_df

# 示例：找到和 `730` 最相似的游戏
similar_games_nn = recommend_similar_games_nn(633230)
display(similar_games_nn)

,app_id,title,tags
15349,599140,Graveyard Keeper,
258829,1789480,Marauders,
6129746,629760,MORDHAU,Medieval Multiplayer First-Person Swordplay Go...
10070052,953880,First Class Trouble,Social Deduction Funny Survival Difficult Mult...
11056140,773790,Ostriv,Simulation City Builder Management Strategy Ec...
11120248,35450,Red Orchestra 2: Heroes of Stalingrad with Ris...,Realistic World War II FPS Multiplayer Histori...
15186422,1254120,Bless Unleashed,RPG Free to Play MMORPG Open World Multiplayer...
15203976,282800,100% Orange Juice,Anime Board Game Cute Multiplayer Strategy Car...
17014803,311260,The Guild 3,Life Sim Economy Medieval Simulation Strategy ...
36580614,2056560,Rock Life: The Rock Simulator,Idler Sandbox Simulation VR 3D Agriculture Fir...


In [70]:
# 🚀 **评估模型**
score, rmse = model.evaluate([test_users, test_games], test_scores_scaled, batch_size=512, verbose=1)
rmse_original = rmse * scaler.scale_[0]

print("Test RMSE:", rmse)
print("Test RMSE (原始单位):", rmse_original)

261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 439.1631 - mae: 12.6897
Test RMSE: 12.686362266540527
Test RMSE (原始单位): 11.219779877563708


In [72]:
print("真实评分最小值:", test_scores.min())
print("真实评分最大值:", test_scores.max())
print("真实评分均值:", test_scores.mean())

真实评分最小值: 0.0
真实评分最大值: 302.68
真实评分均值: 12.801357657880716


In [74]:
from sklearn.metrics import mean_squared_error
print("SVD RMSE:", accuracy.rmse(predictions))
print("KNN RMSE:", accuracy.rmse(knn_predictions))

RMSE: 291.2294
SVD RMSE: 291.22939610672506
RMSE: 26.0750
KNN RMSE: 26.07503153021298
